In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install sentence_transformers
!pip install kss
!pip install transformers

In [3]:
%cd /content/drive/Othercomputers/skt_노트북/ml

/content/drive/Othercomputers/skt_노트북/ml


In [4]:
from finetune import *
train_dataset, eval_dataset = dataset()
finetuning(train_dataset, eval_dataset)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

In [ ]:
##
from tqdm import tqdm
# from konlpy.tag import Mecab
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
import kss
import random
import transformers
from torch.utils.tensorboard import SummaryWriter
import pandas as pd

from load_dataset import *
# from preprocess import CustomTokenizer

from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader

In [ ]:
# ## dataset
# news = NaverSports()
# docs, topics = news.load_data()

In [ ]:
df = pd.read_csv('./newsData/Naver.csv', index_col=0)
df.head()

,contents,titles,categories
0,최 부상 행선지방문 목적 질문에는 묵묵부답 김진방 특파원 북한이 북미 정상회담 ...,동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목,0
1,국회 예산결산특별위원회는 19일 추가경정예산안의 막바지 심사에 돌입했으나 여야 간 ...,"예결위, 추경 막바지 심사 진통…여야 충돌",0
2,안미현 검사 외압 폭로 파문으로 별도 수사단 구성100여일 만에 영장 검찰총장 개입...,외압 논란·항명 사태…산 넘고 물 건넌 권성동 영장 청구,0
3,대표적인 친문재인계인 홍영표 더불어민주당 신임 원내대표가 최근 문 대통령 극렬 지지...,"친문 홍영표, 문빠에 찍혔다…특검 합의에 문자폭탄 공격",0
4,南 맥스선더태영호 등 불만 표시 美 최종타켓조율 안된 의제 반발 北中 관계 개선시진...,"北, 연일 南비난…韓美정상회담 전 경고성 메시지 발신",0


In [ ]:
df.shape

(1581, 3)

In [ ]:
list(df[df['categories'] != 0].index)

[200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366

In [ ]:
##
eval_sents1 = []
eval_sents2 = []
eval_scores = []

train_dataset = []
idxs = list(range(len(df)))
for i in idxs:
    content, title, category = df.loc[i, 'contents'], df.loc[i, 'titles'], df.loc[i, 'categories']
    train_dataset += [InputExample(texts=[content, title], label=1.0)]

    eval_sents1 += [content]
    eval_sents2 += [title]
    eval_scores += [1.0]
    
    new_idxs = list(df[df['categories'] != category].index)
    train_dataset += [InputExample(texts=[content, df.loc[random.choice(new_idxs), 'titles']], label=0.0)]
    eval_sents1 += [content]
    eval_sents2 += [df.loc[random.choice(new_idxs), 'titles']]
    eval_scores += [0.0]


    train_dataset += [InputExample(texts=[df.loc[random.choice(new_idxs), 'contents'], title], label=0.0)]
    eval_sents1 += [df.loc[random.choice(new_idxs), 'contents']]
    eval_sents2 += [title]
    eval_scores += [0.0]

In [ ]:
len(train_dataset)

4800

In [ ]:
# ##
# eval_sents1 = []
# eval_sents2 = []
# eval_scores = []

# train_dataset = []
# idxs = list(range(len(docs)))
# for i in idxs:
#     doc, topic = docs[i], topics[i]
#     train_dataset += [InputExample(texts=[doc, topic], label=1.0)]

#     eval_sents1 += [doc]
#     eval_sents2 += [topic]
#     eval_scores += [1.0]
    
#     new_idxs = idxs[:i] + idxs[i+1:]
#     train_dataset += [InputExample(texts=[doc, topics[random.choice(new_idxs)]], label=0.0)]
#     eval_sents1 += [doc]
#     eval_sents2 += [topics[random.choice(new_idxs)]]
#     eval_scores += [0.0]


#     train_dataset += [InputExample(texts=[doc[random.choice(new_idxs)], topics], label=0.0)]
#     eval_sents1 += [doc[random.choice(new_idxs)]]
#     eval_sents2 += [topic]
#     eval_scores += [0.0]

In [ ]:
# ##
# eval_sents1 = []
# eval_sents2 = []
# eval_scores = []

# train_dataset = []
# idxs = list(range(len(docs)))
# for i in idxs:
#     doc, topic = docs[i], topics[i]
#     train_dataset += [InputExample(texts=[doc, topic], label=1.0)]

#     eval_sents1 += [doc]
#     eval_sents2 += [topic]
#     eval_scores += [1.0]
    
#     new_idxs = idxs[:i] + idxs[i+1:]
#     texts_ = [doc, topics[random.choice(new_idxs)]]
#     train_dataset += [InputExample(texts=texts_, label=0.0)]
#     eval_sents1 += [texts_[0]]
#     eval_sents2 += [texts_[1]]
#     eval_scores += [0.0]

#     texts_ = [doc[random.choice(new_idxs)], topics]
#     train_dataset += [InputExample(texts=texts_, label=0.0)]
#     eval_sents1 += [texts_[0]]
#     eval_sents2 += [texts_[1]]
#     eval_scores += [0.0]

In [ ]:
model = SentenceTransformer("jhgan/ko-sbert-nli")

In [ ]:
import shutil
try:
    shutil.rmtree('./model')
except FileNotFoundError:
    pass

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
train_loss = losses.CosineSimilarityLoss(model)
evaluator = evaluation.EmbeddingSimilarityEvaluator(eval_sents1, eval_sents2, eval_scores)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, evaluator=evaluator, warmup_steps=100, 
          output_path='ko-sbert-navernews')
        #   callback=transformers.integrations.TensorBoardCallback(SummaryWriter()))

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

Iteration:   0%|          | 0/593 [00:00<?, ?it/s]

In [ ]:
model = SentenceTransformer("jhgan/ko-sbert-nli")

In [ ]:
import os
evaluator(model, output_path='model/eval')

0.8096424930489396

In [ ]:
model.save(path='./model')

In [ ]:
mdoel = SentenceTransformer('./model')

In [ ]:
model.save_to_hub()